# 미국(US) 구글 플레이 스토어 리뷰 웹 크롤러
- Contributor: Tony Park
- References
    - Tistory Blog | https://heytech.tistory.com/293
    - Github | https://github.com/park-gb/playstore-review-crawler
- Last Updated@2022-08-24

### 📌 국내(KR) 구글 플레이 스토어 크롤러는 master branch를 참고해 주시길 바랍니다.

# 초기 설정

## 크롬 드라이버 세팅

In [1]:
# chrome_driver = '../chromedriver.exe' # 파일 확장자 이름 표기
chrome_driver = '../chromedriver' # 파일 확장자 이름 미표기

## 수집 앱 URL

In [2]:
URL = 'https://play.google.com/store/apps/details?id=com.roblox.client&hl=en_US&gl=US'

# 패키지 import

In [34]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# 무한 스크롤 함수

In [4]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

# 데이터 크롤링

In [5]:
# 크롬 드라이버 세팅
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(chrome_driver, options = chrome_options)
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인(path 수정 @2022-06-22)
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element(By.XPATH,all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

/var/folders/w1/1zs4c_t97jx0d485bswtnm3m0000gn/T/ipykernel_5639/712538647.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver, options = chrome_options)


스크롤 완료


# HTML Parsing

In [6]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# 크롬 드라이버 종료

In [7]:
driver.quit()

# HTML 데이터 저장

In [8]:
# html 데이터 저장
with open("../dataset/data_html.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

## 데이터프레임 변환

In [35]:
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in tqdm(review_source):
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_txt   = review.find_all(class_ = 'bp9Aid')[0].text
    date_full  = datetime.strptime(date_txt, '%B %d, %Y').strftime('%Y-%m-%d')
    date_year  = datetime.strptime(date_txt, '%B %d, %Y').strftime('%Y') # 연도 데이터 추출
    date_month = datetime.strptime(date_txt, '%B %d, %Y').strftime('%m') # 월 데이터 추출
    date_day   = datetime.strptime(date_txt, '%B %d, %Y').strftime('%d') # 일 데이터 추출

    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][6] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id"       : review_num, 
        "date"     : date_full,
        "date_year" : date_year,
        "date_month": date_month,
        "date_day"  : date_day,
        "rating"   : rating,
        "user_name" : user_name,
        "content"  : content
    }
    dataset.append(data)

100%|██████████████████████████████████████| 2280/2280 [00:03<00:00, 678.97it/s]


In [37]:
dataset[0]

{'id': 1,
 'date': '2022-08-20',
 'dateYear': '2022',
 'dateMonth': '08',
 'dateDay': '20',
 'rating': '5',
 'userName': 'iam_dejjee',
 'content': "Great app. The only things I can complain about is the lag. It wasn't happening but all of a sudden I've become really laggy. Also it disconnects randomly me if my wifi status isn't all the way full or 1 bar away from being full. it only happens when I play the game, so I know it isn't my phone. My wifi will just randomly go off. You have no idea how difficult that makes it to play. Also my screen will sometimes freeze, and sometimes it'll lock too. Please fix it. I really like this game. Ty ❤️❤"}

## 데이터프레임 저장

In [38]:
df = pd.DataFrame(dataset)
df.to_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

# 리뷰 데이터 불러오기

In [39]:
# 저장한 리뷰 정보 불러오기
df = pd.read_csv('../dataset/review_dataset.csv', encoding = 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,2022-08-20,2022,8,20,5,iam_dejjee,Great app. The only things I can complain abou...
1,2,2022-08-01,2022,8,1,5,raesean collins,"This is a pretty cool app,I love how there dif..."
2,3,2022-08-03,2022,8,3,4,Daelynn,"This game is fun, you could play with a bunch ..."
3,4,2022-08-19,2022,8,19,3,Ti,Buggy but it's fun. The camera sometimes bugs ...
4,5,2022-08-22,2022,8,22,4,Deonna Perkins,I love this game i play it all the time. my on...
...,...,...,...,...,...,...,...,...
2275,2276,2022-07-20,2022,7,20,3,Vivaan Bartariya,"This game is great, on other devices. I have b..."
2276,2277,2022-06-26,2022,6,26,4,Eliana Clements,it is amazing! I love that I can serch anythin...
2277,2278,2022-07-18,2022,7,18,5,sofia v,"Roblox, is really nice. all though i am experi..."
2278,2279,2022-06-06,2022,6,6,4,Sandhya Mishra,I love this game. Although it has many lags an...
